In [2]:
import utils

In [3]:
class TensorData:

    def __init__(self, img_path, box_path):

        tensor_dict = utils.data_to_tensor(img_path, box_path)

        self.img = tensor_dict['img']
        self.box = tensor_dict['box']
        self.labels = tensor_dict['labels']